In [ ]:
!git clone https://github.com/Bhuvan-21/gdown.pl.git
!gdown.pl/gdown.pl https://drive.google.com/file/d/1RDSE1uQ8EKvVAwldDbW0jKj8pr5-ro7I/view?usp=sharing 'Data.zip'
!mkdir img_dataset
!cd img_dataset
!unzip Data.zip
!cd ..

Streaming output truncated to the last 5000 lines.
  inflating: Deblur_Data/val/ld/041161.jpg  
  inflating: Deblur_Data/val/ld/042309.jpg  
  inflating: Deblur_Data/val/ld/013709.jpg  
  inflating: Deblur_Data/val/ld/029160.jpg  
  inflating: Deblur_Data/val/ld/051866.jpg  
  inflating: Deblur_Data/val/ld/046926.jpg  
  inflating: Deblur_Data/val/ld/027074.jpg  
  inflating: Deblur_Data/val/ld/016157.jpg  
  inflating: Deblur_Data/val/ld/014481.jpg  
  inflating: Deblur_Data/val/ld/003656.jpg  
  inflating: Deblur_Data/val/ld/048719.jpg  
  inflating: Deblur_Data/val/ld/041717.jpg  
  inflating: Deblur_Data/val/ld/002172.jpg  
  inflating: Deblur_Data/val/ld/024996.jpg  
  inflating: Deblur_Data/val/ld/020198.jpg  
  inflating: Deblur_Data/val/ld/050320.jpg  
  inflating: Deblur_Data/val/ld/033599.jpg  
  inflating: Deblur_Data/val/ld/010301.jpg  
  inflating: Deblur_Data/val/ld/005641.jpg  
  inflating: Deblur_Data/val/ld/007488.jpg  
  inflating: Deblur_Data/val/ld/039624.jpg  
  in

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms


import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
path_hd = "Deblur_Data/train/hd"
path_ld = "Deblur_Data/train/ld"

In [ ]:
img_names = os.listdir(path_hd)

In [ ]:
img_names[0:5]

['001315.jpg', '017554.jpg', '038598.jpg', '007773.jpg', '050092.jpg']

In [ ]:
batch_size = 512

In [ ]:
def dataLoader(img_names, batch_size):
    
    sz = len(img_names)
    
    for i in range(0, sz, batch_size):
        hd = []
        ld = []
        for j in range(batch_size):
            img_hd = plt.imread(path_hd + "/" + img_names[(i+j)%sz])
            img_ld = plt.imread(path_ld + "/" + img_names[(i+j)%sz])
            hd.append(img_hd)
            ld.append(img_ld)
            
        hd = np.array(hd)
        ld = np.array(ld)
        
        hd = np.transpose(hd, (0, 3, 1, 2))
        ld = np.transpose(ld, (0, 3, 1, 2))
        
        hd = torch.from_numpy(hd).float()
        ld = torch.from_numpy(ld).float()
        
        yield hd, ld

In [ ]:
def scale(x, feature_range = (1, -1)):
    
    a, b = feature_range
    x /= 255.0
    x = (a-b)*x + b
    
    return x

def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
def real_loss(D_out, smoothing = False):
    
    batch_size = D_out.size(0)
    labels = torch.ones((batch_size))
    
    if smoothing:
        labels *= 0.9
    
    if train_on_gpu:
        labels = labels.cuda()
    
    criterion = nn.BCEWithLogitsLoss()
    loss = criterion(D_out.squeeze(), labels)
    
    return loss


def fake_loss(D_out):
    
    batch_size = D_out.size(0)
    labels = torch.zeros((batch_size))
    
    if train_on_gpu:
        labels = labels.cuda()
    
    criterion = nn.BCEWithLogitsLoss()
    loss = criterion(D_out.squeeze(), labels)
    
    return loss

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

def conv(in_channels, out_channels, kernel_size, stride = 2, padding = 1, batch_norm = True):
    layers = []
    
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias = False)
    layers.append(conv_layer)
    
    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
        
    return nn.Sequential(*layers)

def t_conv(in_channels, out_channels, kernel_size, stride = 2, padding = 1, batch_norm = True):
    layers = []
    
    t_conv_layer = nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias = False)
    layers.append(t_conv_layer)
    
    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
        
    return nn.Sequential(*layers)

class ResidualBlock(nn.Module):
    def __init__(self, conv_dim):
        super(ResidualBlock, self).__init__()
        self.conv_layer1 = conv(in_channels=conv_dim, out_channels=conv_dim, 
                                kernel_size=3, stride=1, padding=1, batch_norm=True)
        
        self.conv_layer2 = conv(in_channels=conv_dim, out_channels=conv_dim, 
                               kernel_size=3, stride=1, padding=1, batch_norm=True)
        
    def forward(self, x):
        out_1 = F.relu(self.conv_layer1(x))
        out_2 = x + self.conv_layer2(out_1)
        return out_2

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, conv_dim = 16):
        super(Discriminator, self).__init__()
        
        self.conv_dim = conv_dim
        
        self.conv1 = conv(3, conv_dim, 4, batch_norm = False)
        self.conv2 = conv(conv_dim, conv_dim*2, 4)
        self.conv3 = conv(conv_dim*2, conv_dim*2, 4)
        self.conv4 = conv(conv_dim*2, conv_dim*2, 4)
        
        self.fc1 = nn.Linear(conv_dim*2*6*6, 512)
        self.fc2 = nn.Linear(512, 1)
        
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x))
        x = F.leaky_relu(self.conv2(x))
        x = F.leaky_relu(self.conv3(x))
        x = F.leaky_relu(self.conv4(x))
         
        x = x.reshape(-1, self.conv_dim*2*6*6)
        
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [ ]:
D = Discriminator(32)
D

Discriminator(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): Linear(in_features=2304, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
class Generator(nn.Module):
    def __init__(self, conv_dim):
        super(Generator, self).__init__()
        
        self.conv1 = conv(3, conv_dim, 4, batch_norm = False)
        self.conv2 = conv(conv_dim, conv_dim*2, 4)
        self.conv3 = conv(conv_dim*2, conv_dim*4, 3, 1, 1)
        self.conv4 = conv(conv_dim*4, conv_dim*4, 3, 1, 1)
        
        self.res1 = ResidualBlock(conv_dim*4)
        self.res2 = ResidualBlock(conv_dim*4)

        self.t_conv1 = t_conv(conv_dim*4, conv_dim*4, 3, 1, 1)
        self.t_conv2 = t_conv(conv_dim*4, conv_dim*2, 3, 1, 1)
        self.t_conv3 = t_conv(conv_dim*2, conv_dim, 4)
        self.t_conv4 = t_conv(conv_dim, 3, 4, batch_norm = False)
        
        
    def forward(self, x):
        
        x = F.leaky_relu(self.conv1(x))
        x = F.leaky_relu(self.conv2(x))
        x = F.leaky_relu(self.conv3(x))
        x = F.leaky_relu(self.conv4(x))
        
        x = self.res1(x)
        x = self.res2(x)

        x = F.leaky_relu(self.t_conv1(x))
        x = F.leaky_relu(self.t_conv2(x))
        x = F.leaky_relu(self.t_conv3(x))
        x = torch.tanh(self.t_conv4(x))
         
        
        return x


In [ ]:
G = Generator(32)
G

Generator(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res1): ResidualBlock(
    (conv_layer1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_layer2)

In [ ]:
import torch.optim as optim

lr = 0.002
beta1 = 0.5
beta2 = 0.999

D_optim = optim.Adam(D.parameters(), lr, [beta1, beta2])
G_optim = optim.Adam(G.parameters(), lr, [beta1, beta2])


In [ ]:
num_epochs = 51
save_path = "drive/MyDrive/Deblur"
d = 1              
g = 2

print_every = 1
save_every = 3
losses = []

In [ ]:
torch.cuda.is_available()

True

In [ ]:
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    G.cuda()
    D.cuda()
    print('GPU available for training. Models moved to GPU')
else:
    print('Training on CPU.')

GPU available for training. Models moved to GPU


In [ ]:
G.load_state_dict(torch.load(save_path + "/G_11.pt", map_location=torch.device('cuda')))
D.load_state_dict(torch.load(save_path + "/D_11.pt", map_location=torch.device('cuda')))

<All keys matched successfully>

In [ ]:
for epoch in range(11, num_epochs):
    
    for batch_i, (high, low) in enumerate(dataLoader(img_names, batch_size)):
    
        high = scale(high)
        low = scale(low)

        if train_on_gpu:
          low = low.cuda()
          high = high.cuda()

        for e in range(d):
        
            batch_size = high.size(0)

            #print(high.max(), low.max(), high.min(), low.min())
            #print(torch.mean(high), torch.mean(low))
            
            #print(high.max(), low.max(), high.min(), low.min())
            #print(torch.mean(high), torch.mean(low))
            

            D_optim.zero_grad()

            
            

            out = G(low)    
            D_low = D(out)
            D_low_loss = fake_loss(D_low)


            D_high = D(high)
            D_high_loss = real_loss(D_high)

            
            D_loss = D_high_loss + D_low_loss
            D_loss.backward()
            D_optim.step()

        
        for e in range(g):
            
            G_optim.zero_grad()
            
            if train_on_gpu:
                low = low.cuda()
                high = high.cuda()

            out = G(low)
            
                
            D_high = D(out)
            
            G_loss = real_loss(D_high) + torch.mean((high - out)**2) - out.std()
            
            G_loss.backward()
            G_optim.step()
            
        if batch_i%10 == 0:
          print(".", end = "")
          #print(torch.mean(high), torch.mean(out))


    if epoch%print_every == 0:        
        
        losses.append((D_loss.item(), G_loss.item()))
                
        print('Epoch [{:5d}/{:5d}] | D_loss: {:6.4f} | G_loss: {:6.4f}'.format(
                epoch+1, num_epochs, D_loss.item(), G_loss.item()))
        
    if epoch%save_every == 0:
      torch.save(G.state_dict(), save_path + '/G_'+str(epoch+1) +'.pt')
      torch.save(D.state_dict(), save_path + '/D_'+str(epoch+1) +'.pt')
        
    

........Epoch [   12/   51] | D_loss: 1.3700 | G_loss: 0.1050
........Epoch [   13/   51] | D_loss: 1.3780 | G_loss: 0.1249
........Epoch [   14/   51] | D_loss: 1.3845 | G_loss: 0.1441
........Epoch [   15/   51] | D_loss: 1.3887 | G_loss: 0.1382
........Epoch [   16/   51] | D_loss: 1.4034 | G_loss: 0.1414
........Epoch [   17/   51] | D_loss: 1.3911 | G_loss: 0.1575
........Epoch [   18/   51] | D_loss: 1.4063 | G_loss: 0.1528
........Epoch [   19/   51] | D_loss: 1.3709 | G_loss: 0.1641
........Epoch [   20/   51] | D_loss: 1.3905 | G_loss: 0.1583
........Epoch [   21/   51] | D_loss: 1.4030 | G_loss: 0.1066
........Epoch [   22/   51] | D_loss: 1.4046 | G_loss: 0.1389
........Epoch [   23/   51] | D_loss: 1.3799 | G_loss: 0.1766
........Epoch [   24/   51] | D_loss: 1.3808 | G_loss: 0.1558
........Epoch [   25/   51] | D_loss: 1.3880 | G_loss: 0.1440
........Epoch [   26/   51] | D_loss: 1.3710 | G_loss: 0.1410
........Epoch [   27/   51] | D_loss: 1.3902 | G_loss: 0.1349
........

KeyboardInterrupt: ignored